## 계층형 질의
테이블에 계층형 데이터가 존재하는 경우 데이터를 조회하기 위해 사용
    ＊ 계층형 데이터 : 동일 테이블에 계층적으로 상위와 하위 데이터가 포함된 데이터

---

START WITH : 계층 구조 전개의 시작 위치를 지정하는 구문(루트 데이터 지정)

CONNECT BY : 다음에 전개될 자식 데이터를 지정하는 구문 (자식 데이터는 해당 절 조건에 만족해야한다)

PRIOR 자식 = 부모 : 자식 데이터에서 부모 데이터 방향으로 전개하는 순방향 전개

PRIOR 부모 = 자식 : 부모 데이터에서 자식 데이터 방향으로 전개하는 역방향 전개

NOCYCLE : 이미 나타났던 데이터가 전개 중에 다시 나타나는 CYCLE → 런타임 오류 발생 → 이 옵션 사용시 사이클 이후의 데이터는 전개하지 않음

ORDER CIBLINGS BY : 동인한 LEVEL 사이에서 정렬을 수행한다.

WHERE : 모든 전개를 수행한 후에 지정된 조건을 만족하는 데이터만 추출(필터링)

---

ORACLE은 계층형 질의를 사용할 때 다음과 같은 가상 칼럼(Pseudo Column)을 제공한다

 - LEVEL :  루트 데이터면 1, 그 하위 데이터이면 2이다. LEAF 데이터까지 1씩 증가한다.
 - CONNECT_BY_ISLEAF : 전개 과정에서 해당 데이터가 리프 데이터면 1, 그렇지 않으면 0이다.
 - CONNECT_BY_ISCYCLE : 전개 과정에서 자식을 갖는데, 해달 데이터가 조상으로서 존재하면 1, 그렇지 않으면 0이다.
 
     → 조상이란 자신으로부터 루트까지의 경로에 존재하는 데이터를 말하며 CYCLE 옵션을 사용했을 때만 사용할 수 있다.
     

In [ ]:
# 순방향 계층형 질의 결과 출력
SELECT LEVEL, 사원, 관리자, CONNECT_BY_ISLEAF ISLEAF
FROM 사원 START WITH 관리자 IS NULL
CONNECT BY PRIOR 사원 = 관리자;

In [ ]:
# 역방향 계층형 질의 결과 출력
SELECT LEVEL, 사원, 관리자, CONNECT_BY_ISLEAF ISLEAF
FROM 사원 START WITH 사원 = 'D'
CONNECT BY PRIOR 관리자 = 사원;

ORACLE은 계층형 질의를 사용할 때 사용자 편의성을 제공하기 위해서 아래와 같은 함수를 제공한다.
- SYS_CONNECT_BY_PATH : 루트 데이터부터 현재 전개할 데이터까지의 경로를 표시한다.
- CONNECT_BY_ROOT : 현재 전개할 데이터의 루트 데이터를 표시한다. (단항 연산자이다)

In [ ]:
SELECT CONNECT_BY_ROOT 사원 루트사원, SYS_CONNECT_BY_PATH(사원,'/') 경로, 사원, 관리자
FROM 사원
START WITH 관리자 IS NULL
CONNECT BY PRIOR 사원 = 관리자

##  셀프조인
셀프조인이란 동일 테이블 사이의 조인을 말한다. 따라서 FROM 절에 동일 테이블이 두 번 이상 나타난다.

동일한 테이블 이지만 개념적으로는 두 개의 서로 다른 테이블을 사용하는 것과 동일하다.

→ 동일 테이블 사이의 조인을 수행하면 테이블과 칼럼 이름이 모두 동일하기 떄문에 식별을 위해 반드시 별칭(ALIAS)을 사용해야 한다.

In [ ]:
SELECT ALIAS명1.칼럼명 ALIAS명2.칼럼명, ...
FROM 테이블1 ALIAS명1, 테이블2 ALIAS명2
WHERE ALIAS명1.칼럼명2 = ALIAS명2.칼럼명1;

In [ ]:
SELECT WORKER.ID 사원번호, WORKER.NAME 사원명, MANAGER.NAME 관리자명
FROM EMP WORKER, EMP MANAGER
WHERE WORKER.MGR = MANAGER.ID;

In [ ]:
SELECT E1.사원, E1.관리자, E2.관리자 차상위_관리자
FROM 사원 E1, 사원 E2
WHERE E1.관리자 = E2.사원
ORDER BY E1.사원;

# 데이터 상에서 상급 관리자가 없는 최상위 사원은 누락 될 수 있다. 이를 해결하기 위해 OUTER JOIN을 활용한다.

In [ ]:
SELECT E1.사원, E1.관리자, E2.관리자 차상위_관리자
FROM 사원 E1, LEFT OUTER JOIN 사원 E2
ON (E1.관리자 = E2.사원)
ORDER BY E1.사원;

# 위와 같이 수정할 경우 상급 관리자가 없는 최상위 사원도 누락되지 않고 출력된다.